## Imports

In [1]:
from functools import cache
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

In [2]:
day = "12"

In [3]:
with open(f"input_j{day}.txt") as f:
    rows = f.readlines()
    rows = list(map(lambda x: x.replace("\n", ""), rows))
    rows = list(map(lambda x: x.split(), rows))

n = len(rows)

In [4]:
arrangements = [row[0] for row in rows]
conditions = [list(map(lambda x: int(x), row[1].split(","))) for row in rows]

## Part 1

In [5]:
def count_all_configs(arrangement, condition):  # Brut force naïf
    new_configs = {arrangement}

    for i in range(len(arrangement)):
        blocked_new_configs = new_configs.copy()
        for new_config in blocked_new_configs:
            new_configs.add(new_config.replace("?", ".", 1))
            new_configs.add(new_config.replace("?", "#", 1))

    configs = [[len(x) for x in config.split(".") if x != ""] for config in new_configs if "?" not in config]
    return sum(list(map(lambda x: x == condition, configs)))

In [6]:
compteur = 0
for i in tqdm(range(n)):
    compteur += count_all_configs(arrangements[i], conditions[i])

compteur

100%|██████████| 1000/1000 [00:39<00:00, 25.48it/s]


8022

## Part 2

In [7]:
new_arrangements = ["?".join([row[0] for i in range(5)]) for row in rows]
new_conditions = [5*list(map(lambda x: int(x), row[1].split(","))) for row in rows]

In [8]:
def new_count_all_configs(arrangement, condition):  # Version itérative moins naïve

    n = len(arrangement)  # Nb de symboles
    m = len(condition)  # Nb de groupes à atteindre

    # On crée une matrice 3D qui dans la case (i, j, k) donnera le nombre de configurations permettant, en partant de la droite de l'arrangement,
    # d'arriver au symbole i en ayant déjà validé m-j groupes et avec un compteur du groupe actuel à k #
    matrice = [[[0 for symbole in range(n+1)] for chiffre in range(m+1)] for symbole in range(n+1)]

    matrice[n][m][0] = 1  # Aucun symbole ni groupe parcouru avec un compteur à 0 # : une config possible
    matrice[n][m-1][condition[m-1]] = 1  # Aucun symbole parcouru avec le dernier groupe atteint suite à un compteur correspondant : une config

    for symbole in range(n-1, -1, -1):  # Pour chaque symbole de droite à gauche
        for groupe, max_count in enumerate(condition):  # Pour chaque groupe à atteindre (ie non atteint)
            for count in range(max_count+1):  # Compteur de # déjà atteints quand on est sur le symbole
                for character in ['.', '#']:
                    if arrangement[symbole] in [character, '?']:
                        if character == '.' and count == 0:  # Si on tombe sur un . et que le compteur de # est nul
                            matrice[symbole][groupe][count] += matrice[symbole+1][groupe][0]  # On ajoute autant de configs qu'il y en avait avant de bouger du symbole de droite
                        elif character == '.' and condition[groupe] == count:  # Si on tombe sur un . et que le compteur de # est égal au pré-requis du groupe
                            matrice[symbole][groupe][count] += matrice[symbole+1][groupe+1][0] # On ajoute autant de configs qu'il y en avait avant de bouger du symbole de droite
                                                                                               # Mais où l'on avait validé un groupe de moins
                        elif character == '#':  # Si on rajoute un # au compteur
                            matrice[symbole][groupe][count] += matrice[symbole+1][groupe][count+1] # On ajoute autant de configs qu'il y en avait avant de bouger du symbole de droite
                                                                                                   # Mais avec un compteur de # incrémenté de 1

        if arrangement[symbole] != "#":  # Si notre symbole n'est pas un #
            matrice[symbole][m][0] += matrice[symbole+1][m][0]  # Enfin, on ajoute autant de configs qu'il y en avait avant de bouger du symbole de droite
                                                                # Mais où l'on n'a toujours pas atteint un seul groupe

    return matrice[0][0][0]  # Nb de configs une fois le bout atteint, tous les groupes validés et aucun # dans le compteur

In [9]:
compteur = 0
for i in tqdm(range(n)):
    compteur += new_count_all_configs(new_arrangements[i], new_conditions[i])

compteur

100%|██████████| 1000/1000 [00:04<00:00, 216.92it/s]


4968620679637